In [1]:
from operator import itemgetter
from pydantic import BaseModel, Field, model_validator

from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableWithMessageHistory
from langchain_core.messages import HumanMessage, trim_messages
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import ChatOpenAI
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_classic import hub
from langchain_community.chat_message_histories import ChatMessageHistory

In [2]:
prompt = ChatPromptTemplate.from_template(
    '주제 {topic}에 대한 짧은 설명을 해주세요.'
)
model = ChatOpenAI(model='gpt-5-nano', temperature=0)

In [3]:
chain = (
    {'topic':RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

print(chain.invoke('더블딥'))

더블딥(Double Dip)은 경제학에서 쓰이는 용어로, 한 번의 침체를 벗어나 짧은 회복을 보이다가 다시 침체로 떨어지는 현상을 말합니다. 즉, 경기 하강이 두 차례 발생하는 이중 침체를 뜻합니다. 보통 실질 GDP나 고용 지표가 두 차례 연속으로 악화된 구간이 반복될 때 사용합니다. 원하시면 다른 분야(예: 음악, 영상 게임 등)에서의 의미도 간단히 설명해 드리겠습니다.


In [4]:
print(await chain.abatch([{'topic':'더블딥'}, {'topic':'인플레이션'}]))

['더블딥은 경제가 한 차례 회복한 뒤 다시 침체에 빠지는 이중 경기 침체 현상을 말합니다. 두 침체 사이의 회복이 짧거나 미약할 수 있으며, 실업 증가와 내수 위축이 동반될 수 있습니다. 정책적으로는 통화정책 완화나 재정 지출 확대 등으로 회복을 촉진하려는 노력이 필요합니다.', '인플레이션은 일정 기간 동안 물가가 전반적으로 상승하고 화폐의 구매력이 떨어지는 현상입니다. 주로 소비자물가(CPI)로 측정됩니다. 주요 원인은 다음과 같습니다:\n- 수요 견인 인플레이션: 수요가 공급을 초과할 때 가격이 올라감.\n- 비용 인상 인플레이션: 원가 상승(임금, 원자재 등)이 가격에 반영.\n- 기대 인플레이션 및 통화량 증가: 물가 상승 기대나 과도한 화폐 공급이 실제 물가를 올림.\n\n영향은 가계의 실질구매력 감소, 저축의 가치 하락, 불확실성 증가 등이 있으며, 기업의 투자와 고용에도 영향이 있습니다. 보통 중앙은행은 금리 정책으로 물가를 안정시키려 하고, 필요 시 재정 정책이나 공급 측 개혁으로 대응합니다.']


In [5]:
for token in chain.stream({'topic':'더블딥'}):
    print(token, end='', flush=True)

더블딥은 경제가 한 번 침체를 벗어나 짧은 회복기를 보이다가 다시 경기 침체에 빠지는 현상을 말합니다. 즉, 두 차례의 연속 침체가 중간에 짧은 상승 구간으로 갈라져 나타나는 것을 의미합니다. 원인으로는 소비·투자 부진, 실업 증가, 정책 반응의 지연 등이 작용할 수 있습니다.

In [6]:
eng_prompt = ChatPromptTemplate.from_template('이 대답을 영어로 번역해주세요.: {answer}')

In [7]:
composed_chain = (
    {'answer':chain}
    | eng_prompt
    | model
    | StrOutputParser()
)

for token in composed_chain.stream({'topic':'인플레이션'}):
    print(token, end='', flush=True)

Inflation is the phenomenon in which the general level of prices for goods and services rises over a period of time, meaning that the amount of goods you can buy with the same money decreases.

- Measurement: It is usually expressed as an annual rate of price growth and is calculated using tools like the Consumer Price Index (CPI).
- Main causes: Demand-pull inflation, cost-push inflation (rising costs of raw materials or labor), and inflation expectations.
- Impact: Real purchasing power declines; the real value of savings falls, and, conversely, borrowers’ interest burdens can change.
- Policy response: Central banks use monetary policy, such as adjusting interest rates, to aim for price stability. Many countries tend to set their price stability target around 2 percent.

In [8]:
kor_chain = (
    ChatPromptTemplate.from_template('{topic}에 대해 짧은 설명을 해주세요.')
    | model
    | StrOutputParser()
)

eng_chain = (
    ChatPromptTemplate.from_template('{topic}에 대해 짧게 영어로 설명을 해주세요.')
    | model
    | StrOutputParser()
)

parallel_chain = RunnableParallel(kor=kor_chain, eng=eng_chain)
result = await parallel_chain.ainvoke({'topic':'인플레이션'})

print(result['kor'])
print('--'*30)
print(result['eng'])

인플레이션은 물가가 전반적으로 지속적으로 오르는 현상입니다. 돈의 공급이 늘어나거나 수요가 강해지거나 생산비가 올라가면 물가도 함께 오르는 경향이 있습니다. 그 결과 같은 돈으로 살 수 있는 양이 줄어들어 생활비가 올라갑니다. 이를 주로 소비자물가지수(CPI)로 측정하고, 중앙은행은 물가를 안정시키려 금리 정책 등을 이용합니다.
------------------------------------------------------------
Inflation is when prices for most goods and services rise over time, so money buys less. It can happen when demand is strong, costs rise, or the money supply grows faster than the economy.


In [9]:
template = PromptTemplate.from_template('주제 {topic}에 대해 금융 관련 짧은 조언을 해주세요.')
print(template.invoke({'topic':'투자'}))
print(template.invoke('투자'))

text='주제 투자에 대해 금융 관련 짧은 조언을 해주세요.'
text='주제 투자에 대해 금융 관련 짧은 조언을 해주세요.'


In [10]:
chain = (
    template
    | model
    | StrOutputParser()
)
print(chain.invoke({'topic':'더블딥'}))

다음은 더블딥 상황을 염두에 둔 짧은 금융 조언입니다.

- 현금 버퍼 먼저: 비상금 6개월~1년치 생활비에 해당하는 현금성 자산 확보.
- 방어적 배분: 주식 비중은 적당히 축소하고, 고품질 채권과 현금성 자산 비중을 늘려 포트폴리오 방어력 강화.
- 다변화 유지: 주식·채권 외에 금 등 대체자산까지 분산하고 지역도 넓게 분산.
- 비용과 레버리지 관리: 저비용 인덱스펀드 중심으로 운용하고 레버리지는 피하거나 최소화.
- 모니터링과 규칙: 주요 경기지표와 금리 동향을 주시하고, 필요 시 자동 리밸런싱이나 분할 매수로 대응.

참고로 개인 상황에 맞춘 조언은 전문가와 상의하는 것이 좋습니다.


In [11]:
template = ChatPromptTemplate.from_messages(
    [('system', '당신은 유능한 금융 조언가입니다.'),
     ('user', '주제 {topic}에 대해 금융 관련 주언을 해주세요.')]
)
print(template.invoke({'topic':'주식'}))
print(template.invoke('주식'))

messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='주제 주식에 대해 금융 관련 주언을 해주세요.', additional_kwargs={}, response_metadata={})]
messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='주제 주식에 대해 금융 관련 주언을 해주세요.', additional_kwargs={}, response_metadata={})]


In [12]:
template = ChatPromptTemplate.from_messages(
    [('system', '당신은 유능한 금융 조언가입니다.'),
     ('placeholder', '{msgs}')]
)
print(template.invoke({'msgs':[HumanMessage(content='안녕하세요.')]}))

messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요.', additional_kwargs={}, response_metadata={})]


In [13]:
examples = [
    {
        "question": "주식 투자와 예금 중 어느 것이 더 수익률이 높은가?",
        "answer": """
후속 질문이 필요한가요: 네.
후속 질문: 주식 투자의 평균 수익률은 얼마인가요?
중간 답변: 주식 투자의 평균 수익률은 연 7%입니다.
후속 질문: 예금의 평균 이자율은 얼마인가요?
중간 답변: 예금의 평균 이자율은 연 1%입니다.
따라서 최종 답변은: 주식 투자
""",
    } ,
    {
        "question": "부동산과 채권 중 어느 것이 더 안정적인 투자처인가?",
        "answer": """
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권
""",
    },
]

In [14]:
embeddings = HuggingFaceEmbeddings(model='BAAI/bge-m3', model_kwargs={'device':'cuda'}, encode_kwargs={'batch_size':8})

In [15]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embeddings=embeddings,
    vectorstore_cls=Chroma,
    k=1
)

question = '부동산 투자의 장점은 무엇인가?'
selected_examples = example_selector.select_examples({'question':question})

for example in selected_examples:
    for k, v in reversed(example.items()):
        print(f'{k}: {v}')

answer: 
후속 질문이 필요한가요: 네.
후속 질문: 부동산 투자의 위험도는 어느 정도인가요?
중간 답변: 부동산 투자의 위험도는 중간 수준입니다.
후속 질문: 채권의 위험도는 어느 정도인가요?
중간 답변: 채권의 위험도는 낮은 편입니다.
따라서 최종 답변은: 채권

question: 부동산과 채권 중 어느 것이 더 안정적인 투자처인가?


In [16]:
example_prompt = PromptTemplate(
    input_variables=['question', 'answer'],
    template='질문: {question}\n답변: {answer}'
)

prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix='다음은 금융 관련 질문과 답변의 예입니다.:',
    suffix='질문: {question}\n답변: ',
    input_variables=['question']
)

In [17]:
chain = prompt | model
response = chain.invoke({'question':'부동산 투자의 장점은 무엇인가?'})
print(response.content)

답변: 부동산 투자의 대표적 장점은 다음과 같습니다.
- 안정적 현금 흐름: 임대 수익으로 정기적으로 현금이 들어옵니다.
- 자본 이득의 가능성: 시가 상승에 따라 자본 손익이 발생할 수 있습니다.
- 인플레이션 헤지: 물가 상승 시 임대료와 자산 가치가 함께 오르는 경향이 있습니다.
- 담보 대출 및 레버리지: 자산을 담보로 대출을 받아 투자 규모를 키울 수 있습니다.
- 포트폴리오 다각화: 주식·채권 등과의 상관관계가 낮아 전체 포트폴리오의 리스크를 분산시키는 효과가 있습니다.
- 실물 자산의 안정감: 물리적 자산으로 보유한다는 점에서 심리적·실무적 안정감을 줄 수 있습니다.

필요하시면 각 장점에 대한 구체적 예시나 주의점도 함께 정리해 드리겠습니다.


In [18]:
prompt = hub.pull('hardkothari/prompt-maker')
print(prompt)

input_variables=['lazy_prompt', 'task'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hardkothari', 'lc_hub_repo': 'prompt-maker', 'lc_hub_commit_hash': 'c5db8eeefa7be4862a9599b759608dd10ee53f53910838f69abb5ab31c257c2d'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert Prompt Writer for Large Language Models.\n\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['lazy_prompt', 'task'], input_types={}, partial_variables={}, template='Your goal is to improve the prompt given below for {task} :\n--------------------\n\nPrompt: {lazy_prompt}\n\n--------------------\n\nHere are several tips on writing great prompts:\n\n-------\n\nStart the prompt by stating that it is an expert in the subject.\n\nPut instructions at the beginning of the prompt and use ### or to separate the instruction and context \n\nBe specific, descriptive and as deta

In [19]:
class FinancialAdvice(BaseModel):
    setup: str = Field(description='금융 조언 상황을 설정하기 위한 질문')
    advice: str = Field(description='질문을 해결하기 위한 금융 답변')

    @model_validator(mode='before')
    @classmethod
    def question_ends_with_question_mark(cls, values: dict) -> dict:
        setup = values.get('setup', '')
        if not setup.endswith('?'):
            raise ValueError('질문은 ?로 끝나야 합니다.')
        
        return values

In [20]:
parser = PydanticOutputParser(pydantic_object=FinancialAdvice)

prompt = PromptTemplate(
    template='다음 금융 관련 질문에 답변해주세요.\n{format_instructions}\n질문:{query}\n',
    input_variables=['query'],
    partial_variables={'format_instructions':parser.get_format_instructions()}
)

chain = prompt | model | parser

In [21]:
try:
    result = chain.invoke({'query':'부동산에 관련하여 금융 조언을 받을 수 있게 질문하세요. 질문은 ?로 끝나야 합니다.'})
    print(result)
except Exception as e:
    print(f'오류 발생: {e}')

setup='부동산 투자를 고려 중인 직장인으로서, 현재 연소득 7,000만 원, 신용등급 양호, 초기 자본 2,000만 원, 매달 주거 비용으로 120만 원을 지출 중이며 5년 내 주택 매입을 계획하고 있습니다. 금리 상승과 부동산 세금 변화에 대비하여, 적절한 다운페이먼트 비율과 모기지 구조를 어떻게 구성하면 재무 리스크를 최소화하고 장기적으로 유리할까요?' advice='권장 전략은 다음과 같습니다: 1) 다운페이먼트 20% 이상 확보를 목표로 초기 자본 2,000만 원으로 매입가 1억 2,000만 원대 이하의 주택이면 20%를, 더 높은 가격대에서는 25% 정도를 고려합니다. 2) 대출 구조: 금리 상승기에 대비해 고정금리 기간을 5년 이상으로 설정하거나 초기 고정 후 변동형으로 전환하는 혼합형을 선호합니다. 3) 만기 선택: 월상환액과 총이자 비용의 균형을 맞추려 20~25년 만기 구간을 우선 고려합니다. 4) DTI/LTV 관리: 주택담보대출 LTV를 약 60~80%, 총부채상환비율(DTI)을 소득 대비 40% 이하로 관리하는 것이 바람직합니다. 5) 비용 예측: 취득세, 등기비, 보험, 재산세 등 초기 비용과 매월 관리비를 포함한 실질 월지출을 계산합니다. 6) 리스크 관리: 금리 변동에 대비한 시나리오를 작성하고 필요 시 재융자를 통해 이자 부담을 줄이는 방안을 모색합니다. 7) 실행 로드맵: 사전승인 대출 받기, 금융기관 비교, 최적의 금리와 수수료 조건으로 계약하기 등 구체적 단계로 진행합니다. 필요하시면 귀하의 지역과 구체적 예산에 맞춘 수치가 반영된 맞춤 시나리오를 만들어 드리겠습니다.'


In [22]:
prompt = ChatPromptTemplate.from_messages(
    [('system', '당신은 금융 상담사입니다. 사용자에게 최선의 금융 조언을 제공합니다.'),
     ('placeholder', '{msgs}')]
)

chain = prompt | model

In [23]:
chat_history = ChatMessageHistory()

chat_history.add_user_message('저축을 늘리기 위해서 무엇을 할 수 있나요?')
chat_history.add_ai_message('저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요.')
chat_history.add_user_message('방금 뭐라고 했나요?')

ai_response = chain.invoke({'msgs':chat_history.messages})
print(ai_response.content)

방금 드린 말은 다음과 같습니다:  
"저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요."

의미를 간단히 풀면:
- 저축 목표를 구체적으로 정합니다(예: 비상금 3–6개월치, 특정 금액의 여행 자금 등).
- 매달 같은 금액이 자동으로 저축계좌로 들어가도록 자동 이체를 설정합니다.

실행 팁:
- 월급일에 맞춰 자동 이체를 설정하면 잊지 않고 저축이 계속됩니다.
- 비상금은 별도의 저축계좌에 두고 필요 자금이 빠져나가도 손대지 않도록 합니다.
- 상황에 따라 목표 금액과 이체 금액을 조정해도 됩니다.

원하시면 현재 수입, 고정지출, 목표 금액과 기간을 알려주시면, 당신에게 맞춘 구체적인 저축 계획을 같이 만들어 드릴게요.


In [24]:
chat_history = ChatMessageHistory()

prompt = ChatPromptTemplate.from_messages(
    [('system', '당신은 금융 상담사입니다. 모든 질문에 최선을 다해 답변하십시오. 일단은 간략하게 설명해준 후, 같은 내용에 대해서 추가적인 설명을 요청하면 더 자세히 답변하십시오.'),
     ('placeholder', '{chat_history}'),
     ('user', '{question}')]
)

chain = prompt | model

In [25]:
chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key='question',
    history_messages_key='chat_history'
)

In [26]:
result = chain_with_message_history.invoke(
    {'question':'저축을 늘리기 위해 무엇을 할 수 있나요?'},
    {'configurable':{'session_id':'unused'}}
)
print(result.content)

간단히 말해, 수입에서 비용을 깎아 자동으로 저축을 늘리고, 남은 금액을 현명하게 굴리면 저축이 늘어납니다. 핵심은 예산 관리, 자동화, 비상금 마련, 부채 관리, 그리고 수입 확대나 합리적 투자까지 균형 있게 하는 것입니다.

다음이 바로 시작하기 좋은 방법들입니다.
- 예산 잡기 및 지출 줄이기
  - 월 수입-필요지출-저축의 구조를 정하고, 불필요한 지출을 식별해 줄이기.
  - 50/30/20 같은 간단한 규칙이나 자신만의 규칙으로 운영.
- 저축 자동화
  - 월급일에 특정 금액이 자동으로 별도 저축계좌로 빠져나가도록 설정.
  - 저축 계좌를 생활비 계좌와 분리해 '접근성 감소' 효과를 활용.
- 비상금 마련
  - 3~6개월 생활비에 해당하는 금액을 비상금으로 확보해 급작스런 상황에 대비.
- 부채 관리
  - 고금리 부채부터 우선 상환하고, 가능하면 재융자나 더 낮은 이자 상품으로 리팩토링.
- 수입 늘리기
  - 업무 역량 강화, 부업이나 프리랜스 활동 등 수입 다원화를 모색.
- 투자와 자산 배분
  - 장기 목표에 맞춰 위험 허용도에 맞는 포트폴리오 구성. 자금의 일부는 비상금+필요한 경우에 사용할 준비금으로 확보.
- 세제 혜택 및 적합한 금융상품 활용
  - 연금저축, IRP 등 세제 혜택이 있는 계좌를 활용해 저축 효과를 높이기.
- 주기적 점검
  - 매월/분기마다 목표 달성 상황과 지출 구조를 점검하고 필요 시 조정.

원하시면 위 내용을 바탕으로 당신 상황에 맞춘 상세 실행 가이드(예산 템플릿, 자동이체 설정 방법, 비상금 목표액 산정, 우선 상환 순서, 초보자용 투자 포트폴리오 예시 등)를 단계별로 제공해 드리겠습니다. 먼저 몇 가지 정보를 알려주실 수 있을까요?
- 월 수입과 고정지출 규모는 어느 정도인가요?
- 현재 비상금이 있나요? 있다면 어느 정도인가요?
- 빚이 있다면 이자율과 잔액은 어떻게 되나요?
- 저축 목표는 단기(1년 내)인가요, 아니면 장기(5년 이상)인가요?
- 투자에 대한 위험 허용도는 어느 정도인가요? 

In [27]:
result = chain_with_message_history.invoke(
    {'question':'알려준 전략을 바탕으로 저축을 늘려서 집을 살 수 있을까요?'},
    {'configurable':{'session_id':'unused'}}
)
print(result.content)

네, 가능합니다. 다만 목표 집값, 필요한 다운페이, 기간, 현재 재정 상태에 따라 구체 계획이 달라집니다. 아래의 간단한 체크와 실행 가이드를 참고해 보시고, 원하시면 당신 상황에 맞춘 구체 실행안도 만들어 드리겠습니다.

핵심 실행 틀
- 목표 설정
  - 예상 집값 P를 정하고, 다운페이 비율(예: 20% 이상)을 정합니다.
  - 원하는 구매 시점 T(개월 또는 년 단위)를 정합니다.
- 현재 재무 점검
  - 월순수입, 고정지출, 비상금 현황, 남아있는 부채와 이자율을 확인합니다.
- 집자금 전용 계좌 만들기
  - 집자금 목표를 위한 별도 계좌를 만들고, 월급날 자동이체로 고정적으로 입금합니다.
  - 생활비 계좌와 분리해 접근성을 낮추는 효과를 활용합니다.
- 예산과 지출 조정
  - 불필요한 지출을 줄이고, 남는 금액의 일정 비율을 집자금으로 배정합니다.
  - 예산 규칙(예: 50/30/20 등)을 자신에게 맞게 조정합니다.
- 비상금 유지와 우선순위 조정
  - 3~6개월 생활비 규모의 비상금을 우선 확보한 뒤, 남는 자금을 집자금에 집중합니다.
- 고금리 부채 먼저 상환
  - 이자율이 높은 부채가 있다면 먼저 갚아 현금 흐름을 개선합니다.
- 투자 및 운용(짧은 기간의 목표일 때)
  - 1–3년 내 purchasing이 목표라면 원금보전형 상품 위주로 운용하고, 지나치게 공격적인 투자로는 리스크를 피합니다.
- 정기 점검
  - 매월/분기별 진행 상황을 점검하고 필요 시 목표를 재설정합니다.

예시 계산 방식(가이드)
- 목표 가격 P, 다운페이 비율 D%), 기간 T년
- 다운페이 금액 = P × D
- 매월 필요한 저축액 ≈ 남은 다운페이 금액을 남은 기간으로 나눈 값
- 현재 비상금과 기존 저축 여부를 반영해 실제 매월 저축액을 조정합니다.

다음 정보를 주시면 당신 상황에 맞춘 구체 실행 계획(예산 템플릿, 자동이체 설정 방법, 비상금 목표액 산정, 우선 상환 순서, 초보자용 집자금 포트폴리오 예시)을 단계별로 만들어 드리겠습니다

In [28]:
trimmer = trim_messages(strategy='last', max_tokens=3, token_counter=len)

In [29]:
chain_with_trimming = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history') | trimmer)
    | prompt 
    | model
)

In [30]:
chain_with_trimmed_history = RunnableWithMessageHistory(
    chain_with_trimming,
    lambda session_id: chat_history,
    input_messages_key='question',
    history_messages_key='chat_history'
)

In [31]:
result = chain_with_trimmed_history.invoke(
    {'question':'언급해준 내용을 세 문장으로 간략하게 정리해줄 수 있나요?'},
    {'configurable':{'session_id':'finance_session_1'}}
)
print(result.content)

다음과 같이 요약할 수 있습니다:
- 집값 목표와 다운페이 비율, 구매 시점을 정하고, 집자금 전용 계좌를 만들어 월급 자동이체로 저축을 시작합니다.
- 비상금(3-6개월 생활비)을 확보하고 고금리 부채를 먼저 상환하며, 남은 자금은 필요에 따라 보수적 투자로 운용합니다.
- 예산 관리 규칙으로 지출을 줄이고 수입 증가 전략과 세제 혜택 활용 등을 통해 목표를 주기적으로 점검하고 재설정합니다.

더 자세한 실행 가이드를 원하시면 상황에 맞춘 구체안을 드리겠습니다.


In [32]:
def summarize_messages(chain_input):
    stored_messages = chat_history.messages

    if len(stored_messages) == 0:
        return False
    
    summarization_prompt = ChatPromptTemplate.from_messages(
        [('placeholder', '{chat_history}'),
         ('user', '이전 대화를 요약해 주세요. 가능한 많은 세부 정보를 포함하십시오.')]
    )

    summarization_chain = summarization_prompt | model
    summary_message = summarization_chain.invoke({'chat_history':stored_messages})

    chat_history.clear()
    chat_history.add_message(summary_message)

    return True

In [33]:
chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

In [34]:
result = chain_with_summarization.invoke(
    {'question':'저에게 어떤 조언을 해주셨나요?'},
    {'configurable':{'session_id':'unused'}}
)

print(result.content)

간략 요약으로 드렸던 조언은 아래와 같습니다.

- 목표와 방향
  - 집 자금을 늘려 집 구매를 목표로, 예산 관리, 자동저축, 비상금 확보, 고금리 부채 관리, 수입 증가 시도, 투자·세제 혜택을 균형 있게 활용.

- 실행 프레임의 핵심
  - 구체적 목표 설정(예상 집값, 다운페이 비율, 구매 시점) → 재무 상태 점검(수입·지출·비상금·부채·이자) → 집자금 전용 계좌와 자동 이체로 저축 → 예산 조정으로 남는 금액의 일정 비율 집자금에 배정 → 비상금 3–6개월 유지 우선 → 고금리 부채 상환 우선(재융자 검토 가능) → 투자 전략은 기간에 따라 원금보전형 우선으로 조정 → 정기 점검으로 계획 재설정.

- 집 구매 목표를 위한 구체 제안
  - 목표값 설정, 현재 재무 상태 확인, 집자금 전용 계좌 및 자동저축, 지출 축소로 저축액 증가, 비상금 우선 확보, 필요 시 부채 관리 및 투자 전략 조정, 정기 점검.

- 필요 정보와 맞춤화 포인트
  - 목표 집값/다운페이 비율/구매 시점, 월수입/지출, 현재 비상금 규모, 부채 잔액과 이자율, 위험 허용도(보수적/중립적/공격적) 등을 알려주시면 맞춤 실행 계획을 바로 만들어 드립니다.

- 향후 지원 가능 영역
  - 대출 비교 방법, 세제 혜택 활용 방법, 연금저축/IRP 등 세제 우대 상품 활용, 기간별 포트폴리오 예시(보수적/중립적/공격적).

원하시면 지금 상황에 맞춘 상세한 실행 계획을 바로 만들어 드리겠습니다. 현재 가지고 계신 수치나 우선순위(예: 더 빨리 사고 싶다 vs. 안정적으로 천천히 모으고 싶다)를 알려주시면 바로 적용해 드릴게요. 어떤 부분부터 더 자세히 보고 싶으신가요?
